<a href="https://colab.research.google.com/github/ivenk/ma_dl/blob/master/ma_keygen_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
from PIL import Image

In [2]:
class MultiClass(nn.Sequential):
    
    def __init__(self, input_dim=512, hidden_layers=5, output_dim=128):
        super(MultiClass, self).__init__()
        self.l1 = nn.Linear(input_dim, input_dim)
        self.l2 = nn.Linear(input_dim, input_dim)
        self.l3 = nn.Linear(input_dim, input_dim)
        self.l4 = nn.Linear(input_dim, input_dim)
        self.l5 = nn.Linear(input_dim, output_dim)
    

 #   def classifier(self,z):
  #      return nn.functional.softmax(self(z))

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))       
        x = F.relu(self.l4(x))
        x = F.relu(self.l5(x))
        return torch.sigmoid(x)


In [3]:
class CustomFolderDataset(Dataset):
    def __init__(self, path_img_folder, transform=None):
        self.dataset = datasets.ImageFolder(path_img_folder, transform= transform)

In [4]:
class CustomCSVDataset(Dataset):

    def __init__(self, path_img_folder, path_to_csv, transform=None):
        self.meta_data = pd.read_csv(path_to_csv, sep=';')
        self.path_img_folder = path_img_folder
        self.transform = transform


    def __len__(self):
        return len(self.meta_data)


    def __getitem__(self, index):
        name, labels, validation = self.meta_data.iloc[index]
        img = Image.open(self.path_img_folder + '/' + name + '.png')

        labels = labels#.split(sep=" ")
        print(labels)
        sample = { 'image' : img, 'labels' : labels}

        if self.transform:
            sample['image']= self.transform(sample['image'].convert("RGB"))

        return sample

In [5]:
model = MultiClass()
#model.cuda()
#dtype = torch.cuda.FloatTensor

In [6]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [7]:
transform = transforms.Compose([
    scaler, 
    to_tensor,
    normalize
])

In [8]:
batch_size = 200
#dataset = CustomDataset('../../data/lfw.mini.list.out', '../../data/labels.csv', transform=transform)
dataset = datasets.ImageFolder('../../data/lfw.mini.out/train', transform=transform)
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset)

In [9]:
dataset_valid = datasets.ImageFolder('../../data/lfw.mini.out/valid', transform=transform)
dataset_valid.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader_valid = DataLoader(dataset_valid)

In [10]:
# Load the pretrained model
resnet = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = resnet._modules.get('avgpool')

In [11]:
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
def get_vector(img):
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))    
    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)    

    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
      my_embedding.copy_(o.data.squeeze())    
    
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)    
    # 6. Run the model on our transformed image
    resnet(t_img)    
    # 7. Detach our copy function from the layer
    h.remove()    
    # 8. Return the feature vector
    return my_embedding

In [13]:
def get_vector2(img):
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(img)    
    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)    #adjust for batch size here

    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
      my_embedding.copy_(o.data.squeeze())    
    
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)    
    # 6. Run the model on our transformed image
    resnet(t_img)    
    # 7. Detach our copy function from the layer
    h.remove()    
    
    # 8. Return the feature vector
    return my_embedding

In [14]:
def get_target_vector(target):
    data = []

    binary = f'{int(target):0128b}'
    for c in binary:
        data.append(int(c))

    return torch.tensor(data)

In [15]:
from torch.autograd import Variable

In [16]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) #weight_decay=1e-4 


In [17]:
def calculate_mistakes(pred, target):
    mistakes = 0
    
    if len(pred) != len(target):
        raise Exception('sizes of both tensors must match')
        
    for x,y in zip(pred, target):
        if round(x.item()) != y.item():
            mistakes = mistakes + 1
        
    return mistakes

In [18]:
d,_ = dataset[0]
a,_ = dataset[10]

first_vector = get_vector2(d.unsqueeze(0))
second_vector = get_vector2(a.unsqueeze(0))


cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(first_vector.unsqueeze(0),
              second_vector.unsqueeze(0))

print('\nCosine similarity: {0}\n'.format(cos_sim))


    


Cosine similarity: tensor([0.8081])



In [19]:
epochs = 20
for e in range(epochs):
    epoch_mistakes = 0
    epoch_loss = 0
    size = len(loader)
    
    for index, (data, target) in enumerate(loader):
        data_t   = get_vector2(data)
        
        target_t = get_target_vector(dataset.idx_to_class[target[0].item()])        

        data_v   = Variable(data_t, requires_grad=False)
        target_v = Variable(target_t, requires_grad=False)

        # forward
        pred = model.forward(data_v)
        # zero grads
        optimizer.zero_grad()
        # calculate loss
        loss = loss_func(pred, target_v.float())
        epoch_loss = epoch_loss + loss
        # back prop
        loss.backward()
        optimizer.step()
        
        epoch_mistakes = epoch_mistakes + calculate_mistakes(pred, target_t)
        
        #print(str(index), end =" ")
        
        
    valid_loss = 0
    valid_mistakes = 0
    size_valid = len(loader_valid)
    # do validation
    for index, (data, target) in enumerate(loader_valid):
        data_t   = get_vector2(data)
        target_t = get_target_vector(dataset.idx_to_class[target[0].item()])

        data_v   = Variable(data_t, requires_grad=False)
        target_v = Variable(target_t, requires_grad=False)

        # forward
        pred = model.forward(data_v)
        loss = loss_func(pred,target_v.float())
        
        valid_loss = valid_loss + loss
        
        valid_mistakes = valid_mistakes + calculate_mistakes(pred, target_t)
        
    optimizer.zero_grad()
    
    print('=============================')
    print('epoch loss on training set: ' + str(epoch_loss))
    print('epoch loss on validation set:' + str(valid_loss))
    print('average mistakes on validation set: ' + str(valid_mistakes/size_valid))
    

TypeError: softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, name dim, torch.dtype dtype)
 * (Tensor input, int dim, torch.dtype dtype)


In [ ]:
torch.save(model.state_dict(), 'save_v2_20.pt')